# Presynaptic reconstructions

## Setup

Run the following code cell to import the necessary packages and modules. 

The 'ecrest' class will be imported from eCREST_cli.py

In [ ]:
from pathlib import Path
import json
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from igraph import Graph as ig_Graph
from igraph import plot as ig_plot
from scipy.spatial.distance import cdist
from random import choice as random_choice
from itertools import combinations
from numpy import array, unravel_index, argmin, mean,unique,nan
import pandas as pd
from copy import deepcopy
from datetime import datetime
from time import time
import neuroglancer
from webbrowser import open as wb_open
from webbrowser import open_new as wb_open_new
import neuroglancer
from eCREST_cli import ecrest, import_settings

## Import settings

Save a copy of settings_dict.json (found in the "under construction" directory of eCREST repo) locally somewhere outside the repo (like in your save_dir) and change the directory paths to match your setup. Then use the following code cell to import those specs. This avoids needing to re-type the save_dir and db_path each time you "git pull" updates from the repo to this notebook (because everyone has different directory paths).

You will, however, need to change the "path_to_settings_json" before running the code cell if you git pull. 

In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'

settings_dict = import_settings(path_to_settings_json)

## EDIT reconstruction from file

Use the following code cell to launch an editable reconstruction from an existing eCREST json file.

Note that you can also initialize an editable reconstruction using any of the following:
- (segment_id, segment_list): the main_base_id from the neuroglancer file you are converting and a list of base_segments.
- (segment_id): a "main_base_id"
- (filepath): an existing CREST json file

In [ ]:
json_path = Path(settings_dict['save_dir']) / 'todo_presynaptic'
filename = 'cell_graph_387509776__2023-06-09 12.04.01.json'

# This next line launches the reconstruction in neuroglancer
crest = ecrest(settings_dict,filepath= json_path / filename, launch_viewer=True)

'''
Uncomment the following code block to change the key/function pairings so that you do not have to double click to add new segments
'''
# with crest.viewer.config_state.txn() as s:
#     s.input_event_bindings.data_view["alt+mousedown0"]="add-or-remove-seg"
#     s.input_event_bindings.data_view["alt+mousedown2"]="mark-branch-in-colour"
#     print(s.input_event_bindings.data_view)

## check for duplicates

Check if the open reconstruction in progress overlaps with any existing cells in a directory

In [ ]:
directory_to_check = Path(settings_dict['save_dir']) / 'todo_presynaptic'

base_segments = crest.get_base_segments_dict(directory_to_check)
df = crest.check_duplicates(base_segments)
display(df)

## SAVE YOUR WORK BEFORE CLOSING NEUROGLANCER! 

In [ ]:
savedir = Path(settings_dict['save_dir'])

crest.save_cell_graph(directory_path = savedir)

If you want to re-write the file you opened instead of saving with a new timestamp in the filename, you can specify the following function input variables:
- directory_path (= savedir)
- file_name (= current filename for that cell)

Otherwise, the save_cell_graph function automatically creates a new file name with a different timestamp in the name (and you need to manually delete the old one)

# Cell typing

You can check if the reconstruction already has a cell type defined (and what the cell type was defined as).  

In [ ]:
# Assign which method you are using (manual or auto)
method = 'manual'

## Do not edit
crest.get_ctype(method)

If not defined (or defined incorrectly), then define it using the code cell below.
> OPTIONS: mg1, mg2, mgx, lg, lf, lx, mli, gc, gran, sg1, sg2, sgx

After you are finished defining the cell type:  
**DONT FORGET TO SAVE YOUR WORK!**. 

In [ ]:
# Assign the cell type and which method you are using (manual or auto)
cell_type = ''

## Do not edit
method = 'manual'
crest.define_ctype(cell_type,method)

# Dealing with duplicates

## Get segments for main cell and duplicate

In [ ]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)
dirpath = Path(settings_dict['save_dir'])
                  
# First, where is the "main" cell?
main_cell_path = dirpath
    # This will create a base_segments dictionary of all cells in this main directory
cell = ecrest(settings_dict,launch_viewer=False)
base_segments_main =  cell.get_base_segments_dict(main_cell_path)

# Then, where is the "duplicate" cell?
duplicate_cell_path = dirpath / 'todo_presynaptic/'
base_segments_dup = cell.get_base_segments_dict(duplicate_cell_path)


The following code cell will tell you how many segments are overlapping between the two and how many of each cell is missing from the other.

In [ ]:
main = '44098728' 
dup = '42968640'

overlap_segs={}
overlap_segs['main']=base_segments_main[main].difference(base_segments_dup[dup])
overlap_segs['dup']=base_segments_dup[dup].difference(base_segments_main[main])

print(f'{len(overlap_segs["main"])} segments in "main" that are not in dup')
print(f'{len(overlap_segs["dup"])} segments in "dup" that are not in main')

overlap_seg_list = base_segments_main[main] & base_segments_dup[dup]#base_segments_dup[dup]
print(f'{len(overlap_seg_list)} segments in both')


## Create viewer to visualize segments from each cell and their overlap

Red = overlapping segments  
Green = segments only in main cell (not in duplicate)  
Purple = segments only in duplicate cell (not in main)  

In [ ]:
viewer = neuroglancer.Viewer()
viewer.set_state({})

location=[17000,17000,1500]

with viewer.config_state.txn() as s:
    s.show_layer_panel = True ###
with viewer.txn(overwrite=True) as s:
    dimensions = neuroglancer.CoordinateSpace(
        scales=[16, 16, 30],# self.vx_sizes['em'],
        units='nm',
        names=['x', 'y', 'z']   )
    s.showSlices = False
    s.dimensions = dimensions
    s.position = array(location)
    s.layout = "3d"
    s.projectionScale = 30000
    s.projection_background_color= "#000000"

with viewer.txn(overwrite=True) as s:
    wb_open(str(viewer))

db_cursors = sqlite3_connect(settings_dict['db_path'], check_same_thread=False).cursor()
a = ', '.join(['base_address'])
db_cursors.execute(f'''SELECT {a} FROM addresses_table LIMIT 1''')
[base_seg] = db_cursors.fetchall()[0]
two_d_intensity = 0.5

for layer_name in ['main','dup','overlap']:
    with viewer.txn(overwrite=True) as s:
        s.layers[layer_name] = neuroglancer.SegmentationLayer(source = base_seg, segments=[], segment_colors={})
        s.layers[layer_name].ignoreNullVisibleSet = False
        s.layers[layer_name].pick = False
        s.layers[layer_name].selectedAlpha = two_d_intensity #For 2D

### load cells and color overlap
cell_color={'main':'#33cc33','dup':'#cc33ff'}

for k in ['main','dup']:
    with viewer.txn(overwrite=True) as s:
        color_structure = cell_color[k] # blue
        for bs in overlap_segs[k]:
            s.layers[k].segments.add(int(bs))
            s.layers[k].segment_colors[int(bs)] = color_structure # blue

color_structure='#ff0000'
with viewer.txn(overwrite=True) as s:
    for bs in list(overlap_seg_list):
        s.layers['overlap'].segments.add(int(bs))
        s.layers['overlap'].segment_colors[int(bs)] = color_structure # blue

## Load main cell so can add segments from duplicate into main

In [ ]:
dirpath = Path(settings_dict['save_dir']) #/ 'todo_postsynaptic_sg/check-duplicates'
f_list = [f.name for f in dirpath.glob('*' + main + '*')]
try: 
    len(f_list)==1
    main_cell = ecrest(settings_dict,filepath= dirpath / f_list[-1], launch_viewer=True)
except:
    print(f'more than one file for {main}')

In [ ]:
'''
THIS VERSION OF HOW TO DO THIS IS CURRENTLY WORKING BEST
'''
anchor_cell = main_cell
base_ids_added = set()

for base_seg in overlap_segs["dup"].difference(overlap_segs["main"]): #overlap_segs["dup"].difference(overlap_segs["main"]): # dup diff main adds segments in dup that were not in main
    
    if (base_ids_added&set(base_seg)==set()) & (base_seg != anchor_cell.cell_data['metadata']['main_seg']['base']): 
        
        displayed_segs = anchor_cell.assert_segs_in_sync(return_segs=True)
        if base_seg in displayed_segs:
            # print(f'{base_seg} already in cell, continueing')
            continue

        # print(i,base_seg)
        agglo_seg = anchor_cell.get_agglo_seg_of_base_seg(base_seg)

        constituent_base_ids = anchor_cell.get_base_segs_of_agglo_seg(agglo_seg)        
        current_segs = anchor_cell.assert_segs_in_sync(return_segs=True)

        num_base_segs_this_agglo_seg = len(constituent_base_ids)
        constituent_base_ids = [x for x in constituent_base_ids if x not in current_segs]
        constituent_base_ids = [x for x in constituent_base_ids if x not in anchor_cell.cell_data['removed_base_segs']]
        num_base_segs_not_already_included = len(constituent_base_ids)
        
        if len(constituent_base_ids) > anchor_cell.max_num_base_added:
            base_ids = [base_seg]
            # anchor_cell.large_agglo_segs.add(agglo_seg)
            print(f'{len(constituent_base_ids)} other base segments in the agglo segment; max number can add is {anchor_cell.max_num_base_added}')
            # print(f'{base_seg} part of an agglo seg {agglo_seg} that is too large to add, so just adding the one segment')
        else:
            base_ids = constituent_base_ids

        if num_base_segs_this_agglo_seg > num_base_segs_not_already_included:

            if not base_seg in base_ids:
                base_ids.append(base_seg)
        print(base_ids)
        anchor_cell.update_base_locations(base_ids)
        anchor_cell.pr_graph.add_vertices(base_ids)

        if len(base_ids) > 1:
            edges = anchor_cell.get_edges_from_agglo_seg(agglo_seg)
            edges = [x for x in edges if (x[0] in base_ids and x[1] in base_ids)]
            anchor_cell.pr_graph.add_edges(edges)

        join_msg = anchor_cell.add_closest_edge_to_graph(base_ids, base_seg) 
        

        # Update lists of base segments and displayed segs:
        anchor_cell.cell_data['base_segments']['unknown'].update(set(base_ids))

        with anchor_cell.viewer.txn(overwrite=True) as s:

            for bs in base_ids:
                s.layers['base_segs'].segment_colors[int(bs)] = '#d2b48c'
                s.layers['base_segs'].segments.add(int(bs))
                
        base_ids_added.update(base_ids)


        anchor_cell.update_displayed_segs() 
        anchor_cell.assert_segs_in_sync()


## SAVE THE MAIN CELL (with duplicate segments now added)

In [ ]:
anchor_cell.save_cell_graph()